O pymoo oferece **três formas principais** de executar a otimização:

1. **Interface Funcional (`Functional`)** → Mais simples e direta.
2. **Orientado a Objetos com `Next`** → Dá mais controle sobre a execução.
3. **Orientado a Objetos com `Ask` e `Tell`** → Permite modificar e controlar cada iteração.

#  Interface Funcional (Functional)

In [2]:
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.problems import get_problem
from pymoo.optimize import minimize

# Define o problema
problem = get_problem("zdt1")

# Escolhe o algoritmo NSGA-II
algorithm = NSGA2(pop_size=100)

# Executa a otimização
res = minimize(problem,
               algorithm,
               ('n_gen', 10),  # Para após 10 gerações
               seed=1,
               verbose=True)

# Verifica a consistência da solução
print("hash", res.F.sum())

n_gen  |  n_eval  | n_nds  |      igd      |       gd      |       hv     
     1 |      100 |     18 |  1.9687500927 |  2.6048048316 |  0.000000E+00
     2 |      200 |     22 |  1.9687500927 |  2.6551717460 |  0.000000E+00
     3 |      300 |     16 |  1.9156076841 |  2.6054011843 |  0.000000E+00
     4 |      400 |     27 |  1.9156076841 |  2.6075823797 |  0.000000E+00
     5 |      500 |     22 |  1.4885566692 |  2.3893963765 |  0.000000E+00
     6 |      600 |     14 |  1.4885566692 |  1.9295328132 |  0.000000E+00
     7 |      700 |     17 |  1.2814171468 |  2.0349136810 |  0.000000E+00
     8 |      800 |     21 |  1.2543191231 |  2.0440398653 |  0.000000E+00
     9 |      900 |     18 |  1.2044501813 |  1.7727092892 |  0.000000E+00
    10 |     1000 |     15 |  1.2037157959 |  1.6124550984 |  0.000000E+00
hash 41.87514012215159


# Orientado a Objetos (Next Function)

In [3]:
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.problems import get_problem

problem = get_problem("zdt1")
algorithm = NSGA2(pop_size=100)

# Configura o algoritmo sem rodá-lo imediatamente
algorithm.setup(problem, termination=('n_gen', 10), seed=1, verbose=False)

# Roda manualmente cada geração
while algorithm.has_next():
    algorithm.next()
    print(algorithm.n_gen, algorithm.evaluator.n_eval)

# Obtém os resultados
res = algorithm.result()
print("hash", res.F.sum())

2 100
3 200
4 300
5 400
6 500
7 600
8 700
9 800
10 900
11 1000
hash 41.87514012215159


# Orientado a Objetos (Ask and Tell)

In [4]:
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.problems import get_problem

problem = get_problem("zdt1")
algorithm = NSGA2(pop_size=100)

# Configura o algoritmo
algorithm.setup(problem, termination=('n_gen', 10), seed=1, verbose=False)

# Roda manualmente cada geração
while algorithm.has_next():
    pop = algorithm.ask()  # Gera candidatos
    algorithm.evaluator.eval(problem, pop)  # Avalia as soluções
    algorithm.tell(pop)  # Retorna as soluções avaliadas

    print(algorithm.n_gen, algorithm.evaluator.n_eval)

# Obtém o resultado final
res = algorithm.result()
print("hash", res.F.sum())

2 100
3 200
4 300
5 400
6 500
7 600
8 700
9 800
10 900
11 1000
hash 41.87514012215159


# Problema Independente (Problem-Independent Execution)

In [5]:
import numpy as np
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.core.evaluator import Evaluator
from pymoo.core.problem import Problem
from pymoo.core.termination import NoTermination
from pymoo.problems.static import StaticProblem

# Criando um problema sem avaliação definida
problem = Problem(n_var=30, n_obj=2, n_constr=0, xl=np.zeros(30), xu=np.ones(30))

algorithm = NSGA2(pop_size=100)
termination = NoTermination()  # Nunca termina automaticamente

algorithm.setup(problem, termination=termination)

np.random.seed(1)

# Executando manualmente 10 gerações
for n_gen in range(10):
    pop = algorithm.ask()
    X = pop.get("X")

    # Avaliação manual (ZDT1)
    f1 = X[:, 0]
    v = 1 + 9.0 / (problem.n_var - 1) * np.sum(X[:, 1:], axis=1)
    f2 = v * (1 - np.power((f1 / v), 0.5))
    F = np.column_stack([f1, f2])

    static = StaticProblem(problem, F=F)
    Evaluator().eval(static, pop)

    algorithm.tell(pop)
    print(algorithm.n_gen)

res = algorithm.result()
print("hash", res.F.sum())

2
3
4
5
6
7
8
9
10
11
hash 41.87514012215159
